In [1]:
import gym
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from itertools import count

import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available!')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print('MPS is available!')
else:
    device = torch.device('cpu')
    print('CPU...')

MPS is available!
